In [1]:
import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
% matplotlib inline

import glob
import os
import math

from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection

In [2]:
# Compressed Mortality

# get a list of the data files the direction
onlyfiles = [f for f in listdir('data/Compressed Mortality/') if isfile(join('data/Compressed Mortality/', f))]
print(onlyfiles)

# for each txt file...
for i in np.arange(len(onlyfiles)):

    data_temp = pd.read_table('data/Compressed Mortality/' + onlyfiles[i], header='infer', index_col=None,  names=('Notes', 'County', 'County Code', 'Deaths', 'Population', 'Crude Rate'))
    #data_temp.drop(data_temp.index[0])
    data_dropped = data_temp.iloc[1:] #Remove the first row because it's the files's header
    data_dropped['County Code'] = pd.to_numeric(data_dropped['County Code']) # Make the zipcode a float instead of a string
    #print(data_dropped.head(5))

    data_dropped.insert(0, 'year', int(onlyfiles[i][-8:-4])) # Add the year as a column

    if i == 0:
        data = data_dropped
    else:
        data = pd.concat([data, data_dropped]) # add each year's text file to the overall dataframe


data_sorted = data.sort_values(by=['County Code', 'year']) # put in order

#print(data_sorted.head(5))

data_mortality = data_sorted
data_mortality.set_index(['County Code', 'year'], inplace=True, drop =False)

temp = np.zeros((data_mortality.shape[0],1))
unreliable = np.zeros((data_mortality.shape[0],1))
for i in range(data_mortality.shape[0]):
    try:
        temp[i] = float(data_mortality['Crude Rate'].iloc[i].split()[0])
        unreliable[i] = 1
    except:
        temp[i] = float(data_mortality['Crude Rate'].iloc[i])
        unreliable[i] = 0
        
data_mortality['Crude Rate'] =temp


['Compressed Mortality, 1999.txt', 'Compressed Mortality, 2000.txt', 'Compressed Mortality, 2001.txt', 'Compressed Mortality, 2002.txt', 'Compressed Mortality, 2003.txt', 'Compressed Mortality, 2004.txt', 'Compressed Mortality, 2005.txt', 'Compressed Mortality, 2006.txt', 'Compressed Mortality, 2007.txt', 'Compressed Mortality, 2008.txt', 'Compressed Mortality, 2009.txt', 'Compressed Mortality, 2010.txt', 'Compressed Mortality, 2011.txt', 'Compressed Mortality, 2012.txt', 'Compressed Mortality, 2013.txt', 'Compressed Mortality, 2014.txt', 'Compressed Mortality, 2015.txt', 'Compressed Mortality, 2016.txt']


c:\users\steve\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [3]:
# perscrption drugs

# Get all the relevant csv files
onlyfiles = [f for f in listdir('data/Prescriptions/') if isfile(join('data/Prescriptions/', f))]

print(onlyfiles)

# For all the files...
for i in np.arange(len(onlyfiles)):
    # load the CSV
    data_temp = pd.read_csv('data/Prescriptions/' + onlyfiles[i], index_col=None,  names=('State', 'County Code', 'Opioid Perscription Rate'))
    
    # Insert the year as a column in the dataframe
    data_temp.insert(0, 'year', int(onlyfiles[i][-8:-4]))
    if i == 0:
        data = data_temp
    else:
        data = pd.concat([data, data_temp]) # Add to the overall dataframe


data_sorted = data.sort_values(by=['County Code', 'year'])


data_perscriptions = data_sorted

data_perscriptions.set_index(['County Code', 'year'], inplace=True, drop =False)

['rxrate_2006.csv', 'rxrate_2007.csv', 'rxrate_2008.csv', 'rxrate_2009.csv', 'rxrate_2010.csv', 'rxrate_2011.csv', 'rxrate_2012.csv', 'rxrate_2013.csv', 'rxrate_2014.csv', 'rxrate_2015.csv', 'rxrate_2016.csv']


In [7]:
# county health statistics

def create_fips_code(df):
    fips_code = []
    for i in range(df['statecode'].shape[0]):
        # combine the statecode and countycode to create the fips_code
        string = '{:d}{:03d}'.format(df['statecode'].iloc[i], df['countycode'].iloc[i])
        fips_code.append(int(string)) 
    return fips_code

# load all data into a single dataframe
data_dir = 'data/county health rankings'
file_wildcard = os.path.join(data_dir, '*.csv')
for i, filepath in enumerate(reversed(glob.glob(file_wildcard))):
    year = filepath.split('/')[-1].split('\\')[-1].split(' ')[0]
    print(i, filepath, year)
    
    # load the data
    data = pd.read_csv(filepath)
    # convert all columns to lower case
    data.columns = map(str.lower, data.columns)
    # calculate the fips_code and insert as 1st column
    data.insert(loc=0, column='County Code', value=create_fips_code(data))
    # insert the year as the 2nd column
    data.insert(loc=1, column='year', value=int(year))
    #
    if i == 0:
        df = data
        orig_columns = data.columns
    else:
        df = pd.concat([df, data])
    print(data.shape)
    
print(df.shape)

data_health_statistics = df
#data_health_statistics.rename(columns = {'5-digit fips code':'County Code'}, inplace = True)


data_health_statistics = data_health_statistics.sort_values(by=['County Code', 'year'])

data_health_statistics.set_index(['County Code', 'year'], inplace=True, drop =False)

0 data/county health rankings\2017 CHR analytic data.csv 2017
(3186, 358)
1 data/county health rankings\2016 CHR analytic data.csv 2016
(3191, 356)
2 data/county health rankings\2015 CHR analytic data.csv 2015
(3191, 331)
3 data/county health rankings\2014 CHR analytic data.csv 2014
(3191, 326)
(12759, 373)


In [4]:
def create_fips_code(df):
    fips_code = []
    for i in range(df['state fips code'].shape[0]):
        # combine the statecode and countycode to create the fips_code
        string = '{:d}{:03d}'.format(df['state fips code'].iloc[i], df['county fips code'].iloc[i])
        fips_code.append(int(string)) 
    return fips_code

# load the data
data_2016 = pd.read_csv('data/census/2016_census.csv', encoding='utf-8')
data_2015 = pd.read_csv('data/census/2015_census.csv', encoding='utf-8')
print(data_2016.shape)
print(data_2015.shape)
# convert all columns to lower case
data_2016.columns = map(str.lower, data_2016.columns)
data_2015.columns = map(str.lower, data_2015.columns)
# calculate the fips_code and insert as 1st column
data_2016.insert(loc=0, column='County Code', value=create_fips_code(data_2016))
data_2015.insert(loc=0, column='County Code', value=create_fips_code(data_2015))
# insert the year as the 2nd column
data_2016.insert(loc=1, column='year', value=int(2016))
data_2015.insert(loc=1, column='year', value=int(2015))

def replace_substring_in_list(str_list, target_substring, replace_substring):
    new_list = []
    for s in str_list:
        if target_substring in s:
            s = s.replace(target_substring, replace_substring)
        new_list.append(s)
    return new_list

# resolve like-column name mismatches
data_2016.columns = replace_substring_in_list(data_2016.columns, '2016 ', '')
data_2016.columns = replace_substring_in_list(data_2016.columns, 'all people!!', '')
data_2016.columns = replace_substring_in_list(data_2016.columns, 'poverty level!!18 years and over!!', 'poverty level!!')

data_2015.columns = replace_substring_in_list(data_2015.columns, '2015 ', '')
data_2015.columns = replace_substring_in_list(data_2015.columns, 'poverty level!!65 years and over!!', 'poverty level!!')

# concatenate data into a single dataframe
data_census = pd.concat([data_2016, data_2015])
data_census.set_index(['County Code', 'year'], inplace=True, drop=False)

(3220, 1342)
(3220, 1342)


In [20]:
# combine all the data
data = data_mortality.join(data_perscriptions, how='outer', 
                           lsuffix='_data_mortality', rsuffix='_data_perscriptions')
data = data.join(data_health_statistics, how='outer', 
                 lsuffix='', rsuffix='_data_health_statistics')
data = data.join(data_census, how='outer',
                 lsuffix='', rsuffix='_data_census')
data = data.drop(columns=['year'])
data = data.reset_index(level=['year'])

In [22]:
# visual inspection
data.loc[1003].sort_values('year', ascending=False).head(5)

,year,year_data_mortality,Notes,County,County Code_data_mortality,Deaths,Population,Crude Rate,year_data_perscriptions,State,...,percent!!year structure built!!total housing units!!built 1970 to 1979,percent!!year structure built!!total housing units!!built 1980 to 1989,percent!!year structure built!!total housing units!!built 1990 to 1999,percent!!year structure built!!total housing units!!built 2000 to 2009,percent!!year structure built!!total housing units!!built 2010 to 2013,percent!!year structure built!!total housing units!!built 2014 or later,state_data_census,state fips code,unnamed: 0,year_data_census
County Code,,,,,,,,,,,,,,,,,,,,,
1003,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,2016,2016.0,NaN,"Baldwin County, AL",1003.0,32,208563,15.3,2016.0,AL,...,10.6,17.0,25.7,30.4,3.2,0.4,1.0,1.0,1.0,2016.0
1003,2015,2015.0,NaN,"Baldwin County, AL",1003.0,43,203690,21.1,2015.0,AL,...,11.3,16.6,25.1,31.8,2.3,0.2,1.0,1.0,1.0,2015.0
1003,2014,2014.0,NaN,"Baldwin County, AL",1003.0,38,199745,19.0,2014.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,2013,2013.0,NaN,"Baldwin County, AL",1003.0,22,195147,11.3,2013.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# error checking
data.loc[1003].sort_values('year', ascending=False)[['year', str.lower('Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female')]].head(5)

,year,"estimate!!citizen, voting age population!!citizen, 18 and over population!!female"
County Code,,
1003,2017,NaN
1003,2016,78940.0
1003,2015,76791.0
1003,2014,NaN
1003,2013,NaN


In [31]:
data.shape

(41162, 1730)